In [1]:
import tmw_api_keys

ORS_api_key = tmw_api_keys.ORS_api_key
OuiBus_api_key = tmw_api_keys.OuiBus_api_key
SkyScanner_api_key = tmw_api_keys.SkyScanner_api_key
Navitia_api_key = tmw_api_keys.Navitia_api_key

# Initiate classes

In [2]:
class journey:
    def __init__(self, _id, steps=[]):
        self.id = _id
        self.label = ''
        self.score = 0
        self.total_distance = 0
        self.total_duration = 0
        self.total_price_EUR = 0
        self.total_gCO2 = 0
        self.steps = steps
    
    def add(self, steps=[]):
        self.steps.append(steps)
    
    def to_json(self):
        json = {'id':self.id,
                'label':self.label,
                'score':self.score,
                'total_distance':self.total_distance,
                'total_duration':self.total_duration,
                'total_price_EUR':self.total_price_EUR,
                'total_gCO2':self.total_gCO2,
                'journey':[step.to_json() for step in self.steps]
                }
        return json
    
    def reset(self):
        self.score = 0
        self.total_distance = 0
        self.total_duration = 0
        self.total_price_EUR = 0
        self.total_gCO2 = 0
    
    def update(self):
        self.score = 0
        self.total_distance = sum(filter(None,[step.distance_m for step in self.steps]))
        self.total_duration = sum(filter(None,[step.duration_s for step in self.steps]))
        self.total_price_EUR = sum(filter(None,[sum(step.price_EUR) for step in self.steps]))
        self.total_gCO2 = sum(filter(None,[step.gCO2 for step in self.steps]))
    
    def plot_map(self, center=(48.864716,2.349014), tiles = 'Stamen Toner', zoom_start = 10):
        import folium
        map_params = {'tiles':tiles,
              'location':center,
              'zoom_start': zoom_start}
        _map = folium.Map(**map_params)

        for step in self.steps:
            try:
                step.plot_map(center=center, _map=_map)
            except:
                print('ERROR plot map: step id: {} / type: {}'.format(step.id, step.type))
        return _map
    
class journey_step:
    def __init__(self, _id, _type, label='', distance_m=0, duration_s=0, price_EUR=[0.0], gCO2 = 0, geojson=''):
        self.id = _id
        self.type = _type
        self.label = label
        self.distance_m = distance_m
        self.duration_s = duration_s
        self.price_EUR = price_EUR
        self.gCO2 = gCO2
        self.geojson = geojson
        
    def to_json(self):
        json = {'id':self.id,
                 'type':self.type,
                 'label':self.label,
                 'distance_m':self.distance_m,
                 'duration_s':self.duration_s,
                 'price_EUR':self.price_EUR,
                 'gCO2':self.gCO2,
                 'geojson':self.geojson,
                }
        return json
    
    def init_map(self, center, tiles = 'Stamen Toner', zoom_start = 10):
        import folium
        map_params = {'tiles':tiles,
              'location':center,
              'zoom_start': zoom_start}
        _map = folium.Map(**map_params)
        return _map
    
    def plot_map(self, center=(48.864716,2.349014), _map=None):
        import folium
        _map = self.init_map(center) if _map == None else _map
        
        folium.features.GeoJson(data=self.geojson,
                        name=self.label,
                        overlay=True).add_to(_map)
        return _map


# Basic Functions

In [3]:
def geocode_address(address):
    '''
    address: string
    coord : [lon, lat]
    '''
    ORS_client = start_ORS_client()
    coord = ORS_client.pelias_search(address,size=1)['features'][0]['geometry']['coordinates']
    return coord

def create_query(start, to, datetime=''):
    json_query = {
        'query':{
            'start':{
                'address':start,
                'coord':geocode_address(start),  # [lon,lat]
            },
            'to':{
                'address':to,
                'coord':geocode_address(to),
            },
            'datetime':datetime          # example of format (based on navitia): 20191012T063700
            } 
    }
    return json_query

# OpenRouteService Functions

In [4]:
def start_ORS_client():
    import openrouteservice
    ORS_client = openrouteservice.Client(key=ORS_api_key) # Specify your personal API key
    return ORS_client

def ORS_profile(profile):
    dict_ORS_profile = {
        "driving-car":"car", 
        "driving-hgv":"", 
        "foot-walking":"walk",
        "foot-hiking":"walk", 
        "cycling-regular":"bike", 
        "cycling-road":"bike",
        "cycling-mountain":"bike",
        "cycling-electric":"bike"
    }
    return dict_ORS_profile[profile]

def ORS_query_directions(query, profile='driving-car', _id=0, geometry=True):
    '''
    profile= ["driving-car", "driving-hgv", "foot-walking","foot-hiking", "cycling-regular", "cycling-road","cycling-mountain",
    "cycling-electric",]
    '''
    from openrouteservice import convert
    
    ORS_client = start_ORS_client()
    coord = [query['query']['start']['coord'], query['query']['to']['coord'] ]
    ORS_step = ORS_client.directions(
        coord,
        profile=profile,
        instructions=False,
        geometry=geometry,
        )
    
    geojson = convert.decode_polyline(ORS_step['routes'][0]['geometry'])

    step = journey_step(_id, 
                        _type=ORS_profile(profile),
                        label=profile, 
                        distance_m=ORS_step['routes'][0]['summary']['distance'], 
                        duration_s=ORS_step['routes'][0]['summary']['duration'],
                        price_EUR=[0],
                        gCO2=0,
                        geojson=geojson,
                        )
    return step

# Navitia Functions

In [5]:
def start_navitia_client():
    from navitia_client import Client
    navitia_client = Client(user=Navitia_api_key)
    return navitia_client

def navitia_query_directions(query, _id=0):
    navitia_client = start_navitia_client()
    coord = [query['query']['start']['coord'], query['query']['to']['coord'] ]
    
    url = 'coverage/fr-idf/journeys?from={};{}&to={};{}'.format(coord[0][0],coord[0][1],coord[1][0],coord[1][1])
    urm = url + '&data_freshness=base_schedule&max_nb_journeys=3'
    step = navitia_client.raw(url, multipage=False)
 
    return step.json()


In [6]:
"""
https://doc.navitia.io/#journeys
type = 'waiting' / 'transfer' / 'public_transport' / 'street_network' / 'stay_in' / crow_fly
"""
def navitia_journeys(json, _id=0):
    # all journeys loop
    lst_journeys = list()
    for journey in json['journeys']:
        i = _id
        
        # journey loop
        lst_sections = list()
        for section in journey['sections']:

            try:
                lst_sections.append(navitia_journeys_sections_type(section, _id=i))
            except:
                print('ERROR : ')
                print('id: {}'.format(i))
                print(section)
            i = i+1
        lst_journeys.append(lst_sections)
    return lst_journeys

def navitia_journeys_sections_type(json, _id=0):
    switcher_journeys_sections_type = {
        'public_transport': navitia_journeys_sections_type_public_transport,
        'street_network': navitia_journeys_sections_type_street_network,
        'waiting': navitia_journeys_sections_type_waiting,
        'transfer': navitia_journeys_sections_type_transfer,
    }
    func = switcher_journeys_sections_type.get(json['type'], "Invalid navitia type")
    step = func(json, _id)
    return step

def navitia_journeys_sections_type_public_transport(json, _id=0):
    display_information = json['display_informations']
    label = '{} {} / {} / direction: {}'.format(
        display_information['physical_mode'],
        display_information['code'],
        display_information['name'],
        display_information['direction'],
    )
    step = journey_step(_id, 
                        _type=display_information['network'].lower(),
                        label=label, 
                        distance_m=None, 
                        duration_s=json['duration'],
                        price_EUR=[0],
                        gCO2=json['co2_emission']['value'],
                        geojson=json['geojson'],
                        )
    return step

def navitia_journeys_sections_type_street_network(json, _id=0):
    mode = json['mode']
    mode_to_type = {
        'walking':'walk',
        'bike':'bike',
        'car':'car',
    }
    label = '{} FROM {} TO {}'.format(
        mode_to_type[mode],
        json['from']['name'],
        json['to']['name'],
    )
    step = journey_step(_id, 
                        _type=mode_to_type[mode],
                        label=label, 
                        distance_m=None, 
                        duration_s=json['duration'],
                        price_EUR=[0],
                        gCO2=json['co2_emission']['value'],
                        geojson=json['geojson'],
                        )
    return step

def navitia_journeys_sections_type_transfer(json, _id=0):
    mode = json['transfer_type']
    mode_to_type = {
        'walking':'walk',
        'bike':'bike',
        'car':'car',
    }
    label = '{} FROM {} TO {}'.format(
        mode_to_type[mode],
        json['from']['name'],
        json['to']['name'],
    )
    step = journey_step(_id, 
                        _type=mode_to_type[mode],
                        label=label, 
                        distance_m=None, 
                        duration_s=json['duration'],
                        price_EUR=[0],
                        gCO2=json['co2_emission']['value'],
                        geojson=json['geojson'],
                        )
    return step

def navitia_journeys_sections_type_waiting(json, _id=0):
    step = journey_step(_id, 
                        _type='wait',
                        label='wait', 
                        distance_m=None, 
                        duration_s=json['duration'],
                        price_EUR=[0],
                        gCO2=json['co2_emission']['value'],
                        geojson='',
                        )
    return step

def navitia_journeys_correct(journey, json):
    try:
        if type(j) == journey
    except:
        print('ERROR function navitia_journeys_correct() - INPUT Not journey class')
        return False
    
    return journey

# Testing

In [7]:
q = create_query('13 rue Abel','43 Quai Branly')
q

{'query': {'start': {'address': '13 rue Abel', 'coord': [2.37549, 48.847782]},
  'to': {'address': '43 Quai Branly', 'coord': [2.297232, 48.861317]},
  'datetime': ''}}

In [30]:
ORS_query_directions(q,geometry=True)#.plot_map(center=(48.864716,2.349014))

In [9]:
navitia_response = navitia_query_directions(q)

In [10]:
def navitia_journey_desc(json, _id=0):
    fare = json['fare']
    print('WARNING Navitia: Journey -->No fare found') if fare['found'] == False else True
    price_EUR = fare['total']['value']/1000
    
    json_step = {'id':_id,
                 'type':'wait',
                 'label':'wait',
                 'distance_m':None,
                 'duration_s':json['duration'],
                 'price_EUR':[price_EUR],
                 'gCO2':json['co2_emission']['value'],
                }
    return json_step

navitia_response['journeys'][0]#['distance']

{'status': '',
 'distances': {'taxi': 0,
  'car': 0,
  'walking': 905,
  'bike': 0,
  'ridesharing': 0},
 'links': [{'href': 'https://api.navitia.io/v1/coverage/fr-idf/journeys?from=2.37549%3B48.847782&last_section_mode%5B%5D=walking&is_journey_schedules=True&direct_path=none&to=2.297232%3B48.861317&min_nb_journeys=5&min_nb_transfers=1&allowed_id%5B%5D=stop_point%3A0%3ASP%3A59232&allowed_id%5B%5D=stop_point%3A0%3ASP%3A59233&allowed_id%5B%5D=stop_point%3A0%3ASP%3A59249&allowed_id%5B%5D=stop_point%3A0%3ASP%3A59500&first_section_mode%5B%5D=walking',
   'type': 'journeys',
   'rel': 'same_journey_schedules',
   'templated': False}],
 'tags': ['walking', 'ecologic'],
 'nb_transfers': 1,
 'durations': {'taxi': 0,
  'walking': 1061,
  'car': 0,
  'ridesharing': 0,
  'bike': 0,
  'total': 2309},
 'arrival_date_time': '20191013T123133',
 'calendars': [{'active_periods': [{'begin': '20191013', 'end': '20191021'}],
   'week_pattern': {'monday': False,
    'tuesday': False,
    'friday': False,
  

In [11]:
#navitia_response['journeys'][0]['sections'][2].keys()#['mode']#.keys()
navitia_journeys(navitia_response)
navitia_journeys_sections_type_public_transport(navitia_response['journeys'][0]['sections'][1]).to_json()
#navitia_response['journeys'][0]['sections'][1]['geojson']

{'id': 0,
 'type': 'metro',
 'label': 'Métro 1 / Château de Vincennes - La Défense / direction: La Défense (Grande Arche) (Puteaux)',
 'distance_m': None,
 'duration_s': 900,
 'price_EUR': [0],
 'gCO2': 16.707,
 'geojson': {'type': 'LineString',
  'properties': [{'length': 5569}],
  'coordinates': [[2.373449, 48.84556],
   [2.369219, 48.852976],
   [2.361334, 48.855134],
   [2.352074, 48.857356],
   [2.347933, 48.85857],
   [2.340973, 48.86088],
   [2.336574, 48.862372],
   [2.329095, 48.86478],
   [2.321194, 48.865678],
   [2.314123, 48.867744],
   [2.310253, 48.86901]]}}

In [29]:
steps = navitia_journeys(navitia_response)[0]
j = journey(0, steps=steps)
j.update()
j.to_json()


{'id': 0,
 'label': '',
 'score': 0,
 'total_distance': 0,
 'total_duration': 2309,
 'total_price_EUR': 0,
 'total_gCO2': 19.134,
 'journey': [{'id': 0,
   'type': 'walk',
   'label': 'walk FROM 13 Rue Abel (Paris) TO Gare de Lyon (Paris)',
   'distance_m': None,
   'duration_s': 296,
   'price_EUR': [0],
   'gCO2': 0.0,
   'geojson': {'type': 'LineString',
    'properties': [{'length': 331}],
    'coordinates': [[2.3755705056, 48.8477243569],
     [2.3755705056, 48.8477243569],
     [2.375305, 48.847347],
     [2.375358, 48.847322],
     [2.375336, 48.847278],
     [2.37531, 48.847236],
     [2.375193, 48.847105],
     [2.375143, 48.847033],
     [2.375063, 48.846914],
     [2.374414, 48.845987],
     [2.374327, 48.845977],
     [2.373967, 48.845936],
     [2.373751, 48.845861],
     [2.373792, 48.845464],
     [2.3734453503, 48.8455442309],
     [2.373449, 48.84556]]}},
  {'id': 1,
   'type': 'metro',
   'label': 'Métro 1 / Château de Vincennes - La Défense / direction: La Défense (G

In [13]:
steps = navitia_journeys(navitia_response)[0]
steps[1].plot_map()

In [28]:
navitia_response[0]

KeyError: 0